In [2]:
import pandas as pd
from IPython.display import display, HTML
display(HTML("<style>.container{width: 100%}</style>"))

In [3]:
df_raw = pd.read_parquet("./.data/transactions.parquet")
dC_raw = pd.read_csv("./.data/clients.csv")
dP_raw = pd.read_csv("./.data/plants.csv")
dM_raw = pd.read_csv("./.data/materials.csv")

In [29]:
def process_transactions(df):
    df = df[df["sales_sum"] >= 0]
    chq_sum = df.groupby("chq_id").agg(
        chq_sum=pd.NamedAgg("sales_sum", "sum")
    ).reset_index()
    mat_stat = df.groupby(["material", "plant", "chq_date"]).agg(
        chq_count=pd.NamedAgg("chq_id", "nunique"),
        clients_count=pd.NamedAgg("client_id", "nunique"),
        chq_sum=pd.NamedAgg("chq_sum", "sum"),
        sales_count=pd.NamedAgg("sales_count", "sum"),
        sales_sum=pd.NamedAgg("sales_sum", "sum"),
        promo_count=pd.NamedAgg("is_promo", "sum"),
    ).reset_index()
    cli_stat = df.groupby(["client_id", "plant", "chq_date"]).agg(
        chq_count=pd.NamedAgg("chq_id", "nunique"),
        chq_sum=pd.NamedAgg("chq_sum", "sum"),
        items_count=pd.NamedAgg("sales_count", "count"),
        promo_count=pd.NamedAgg("is_promo", "sum"),
    ).reset_index()
    df = pd.merge(chq_sum, df, how='inner', on="chq_id")
    df = pd.merge(mat_stat["material", "plant", "chq_date", "chq_count"], df, how='inner', on=["material", "plant", "chq_date"])
    return df
_df_raw = df_raw.head(40000)
_df = process_transactions(_df_raw)
df = process_transactions(df_raw)

KeyError: "Column 'chq_sum' does not exist!"

In [20]:
_df_raw.shape, _df.shape

((40000, 9), (39972, 10))

In [5]:
display(dP_raw.shape, dC_raw.shape, dM_raw.shape, df.shape)
display(dP_raw.head())
display(dC_raw.head())
display(dM_raw.head())
display(df_raw.head())

(388, 3)

(99995, 4)

(105609, 8)

(32086641, 10)

,plant,plant_type,city
0,95b09698fda1f64af16708ffb859eab9,HM,St. Petersburg
1,926abae84a4bd33c834bc6b981b8cf30,HM,St. Petersburg
2,ae2bac2e4b4da805d01b2952d7e35ba4,HM,St. Petersburg
3,0e7e3cf0ded4d9db8b376b317c007f99,HM,St. Petersburg
4,540bd55a2cf295b8ea9cd78650e89d03,HM,St. Petersburg


,client_id,gender,city,birthyear
0,1a47d62dddacc03fe90c15652f7ae1a4,M,Other,1990.0
1,9cb909f701d25d548e953bff81192b56,F,Other,1969.0
2,d5da0f5b841b4f83383202807995027a,M,Other,1976.0
3,13ed7f16810b17b8cee6de834ac79a48,F,Moscow,1966.0
4,215fe3ea7d5bf0415e5504e2a7f33551,F,Other,1988.0


,material,hier_level_1,hier_level_2,hier_level_3,hier_level_4,vendor,is_private_label,is_alco
0,35cbdf61de9e19c8b417327aaef14c88,NONFOOD,ea5d2f1c4608232e07d3aa3d998e5135,99cad265a1768cc2dd013f0e740300ae,9eed45f71360b4b1e2590637467220e5,212a38db0ddcd009f1e164cc8483485c,0,0
1,c0b0bf24d4ec71da3d304f761ec555d8,NONFOOD,ea5d2f1c4608232e07d3aa3d998e5135,99cad265a1768cc2dd013f0e740300ae,de2eb747e0896c050905a6b635ab800a,9b0b6c7d55413ad3b67761b7b125b534,0,0
2,f0fc5e654a81a7c4b8ba8d7c26546e14,NONFOOD,ea5d2f1c4608232e07d3aa3d998e5135,99cad265a1768cc2dd013f0e740300ae,8a969031832c535daf96e0c2aed8e814,9b0b6c7d55413ad3b67761b7b125b534,0,0
3,353693e64fb5f9e2d29746d7fe6edf1e,NONFOOD,ea5d2f1c4608232e07d3aa3d998e5135,b58f7d184743106a8a66028b7a28937c,4c0dc012ebb679a18b244c53c6f59b5a,a3c8be149d718771e892619bd310b961,0,0
4,5d9e0b4302ce95448cae72165ff4cf5b,NONFOOD,ea5d2f1c4608232e07d3aa3d998e5135,b58f7d184743106a8a66028b7a28937c,4c0dc012ebb679a18b244c53c6f59b5a,a3c8be149d718771e892619bd310b961,0,0


,chq_id,plant,chq_date,chq_position,client_id,material,sales_count,sales_sum,is_promo
0,ce5b3d1c531348c5d0d8e859b6d0cf91,7cd86ecb09aa48c6e620b340f6a74592,2016-11-01,004,90fca68123e692a005a4edeadd94b2f2,232cfe8b28970434f685b737737b26d4,2.0,146.98,0
1,ce5b3d1c531348c5d0d8e859b6d0cf91,7cd86ecb09aa48c6e620b340f6a74592,2016-11-01,003,90fca68123e692a005a4edeadd94b2f2,12ec64451b8e426db16f3f05ea770901,1.0,249.99,1
2,ce5b3d1c531348c5d0d8e859b6d0cf91,7cd86ecb09aa48c6e620b340f6a74592,2016-11-01,012,90fca68123e692a005a4edeadd94b2f2,1212ff8ecf53aebef093ca8ae216db68,1.0,47.99,1
3,ce5b3d1c531348c5d0d8e859b6d0cf91,7cd86ecb09aa48c6e620b340f6a74592,2016-11-01,013,90fca68123e692a005a4edeadd94b2f2,1212ff8ecf53aebef093ca8ae216db68,1.0,47.99,1
4,ce5b3d1c531348c5d0d8e859b6d0cf91,7cd86ecb09aa48c6e620b340f6a74592,2016-11-01,011,90fca68123e692a005a4edeadd94b2f2,571c6d3b559db8445e05265649eb30ab,2.0,53.98,1


In [26]:
def material_stat(df):
    return df.groupby(["material", "plant", "chq_date"]).agg(
        chq_count=pd.NamedAgg("chq_id", "nunique"),
        clients_count=pd.NamedAgg("client_id", "nunique"),
        chq_sum=pd.NamedAgg("chq_sum", "sum"),
        sales_count=pd.NamedAgg("sales_count", "sum"),
        sales_sum=pd.NamedAgg("sales_sum", "sum"),
        promo_count=pd.NamedAgg("is_promo", "sum"),
    ).reset_index()

def client_stat(df):
    return df.groupby(["client_id", "plant", "chq_date"]).agg(
        chq_count=pd.NamedAgg("chq_id", "nunique"),
        chq_sum=pd.NamedAgg("chq_sum", "sum"),
        items_count=pd.NamedAgg("sales_count", "count"),
        promo_count=pd.NamedAgg("is_promo", "sum"),
    ).reset_index()
_dMstat = material_stat(_df)
_dCstat = client_stat(_df)
# _dstat = mk_agg(_df)
display(_dMstat.sort_values("chq_count", ascending=False).head())
display(_dCstat.sort_values("chq_sum", ascending=False).head())
_tmp = pd.merge(_df, _dMstat[["material", "plant", "chq_date", "chq_count"]], how='inner', on=["material", "plant", "chq_date"])
display(_tmp.head())

,material,plant,chq_date,chq_count,clients_count,chq_sum,sales_count,sales_sum,promo_count
24115,b75147feae4cf8ec07fb5f2cef855def,6bcf53c2bdbee891ffaca1e612b0076c,2016-11-02,21,18,21663.0,24.0,83.24,1
28409,d6adc9e7704e7060824cb7da075bf566,6950aac2d7932e1f1a4c3cf6ada1316e,2016-11-02,20,20,51338.0,37.0,246.42,0
7393,3afa2c41341548499934a52d7338f33e,c76b3d3ab1522b66814253d642932e9e,2016-11-02,18,14,43384.0,38.0,257.53,0
28413,d6adc9e7704e7060824cb7da075bf566,7cd86ecb09aa48c6e620b340f6a74592,2016-11-03,16,15,46527.0,31.0,210.46,0
24142,b75147feae4cf8ec07fb5f2cef855def,857778a20b9a41d4ca0d687a36e4bfa8,2016-11-02,15,14,16217.0,17.0,59.16,1


,client_id,plant,chq_date,chq_count,chq_sum,items_count,promo_count
852,3ce736248295e620de4f57d7bbc5f63e,e90b327c1ceca11f1a44a8bcf58af34c,2016-11-01,1,1840676.0,148,96
3562,ff1906e50f019dbadba2a2e867d8049b,b6fb522815d06fed82b0140be4c74680,2016-11-01,1,1437135.0,105,50
2891,ccfa666b62fb7b1f8979607378b697a6,171293841d3fdd4af2e12426ce202ac9,2016-11-01,1,1003000.0,85,31
1747,7d874abadeb27bc8149b8b67bec82ddf,ab1048060528a978a536ee2e9538de28,2016-11-01,3,778162.0,74,20
1874,87b852fea84def055965610c909d861d,b323b8172b421577bf8e70d411d79a50,2016-11-03,2,774216.0,93,57


,chq_id,chq_sum,plant,chq_date,chq_position,client_id,material,sales_count,sales_sum,is_promo,chq_count
0,0013f6dc27537aae9248a1fdf7028860,1762.0,785ee201dc63daae3aecb2c7cbae60d3,2016-11-01,007,24b84e8ab0d7892d12752a9e909f4780,4bd6a36d2865c70fd9a76604f15930db,1.0,58.39,0,1
1,0013f6dc27537aae9248a1fdf7028860,1762.0,785ee201dc63daae3aecb2c7cbae60d3,2016-11-01,015,24b84e8ab0d7892d12752a9e909f4780,f03f28b8ae17d81a590e97d51a65561d,1.0,69.99,1,1
2,0013f6dc27537aae9248a1fdf7028860,1762.0,785ee201dc63daae3aecb2c7cbae60d3,2016-11-01,014,24b84e8ab0d7892d12752a9e909f4780,6485a4dc13fa076319727edfb3ef57eb,1.0,122.59,0,1
3,0013f6dc27537aae9248a1fdf7028860,1762.0,785ee201dc63daae3aecb2c7cbae60d3,2016-11-01,003,24b84e8ab0d7892d12752a9e909f4780,142d63107737a448dc2dfe2dcd0154b8,1.0,6.79,0,4
4,01a3d5f16c39b59accfa43800ba5dfd7,867.0,785ee201dc63daae3aecb2c7cbae60d3,2016-11-01,003,5d20bceec50429c892bfe0814cd8c943,142d63107737a448dc2dfe2dcd0154b8,1.0,6.79,0,4


In [27]:
_df.shape, _dMstat.shape, _tmp.shape

((39972, 10), (33932, 9), (39972, 11))